In [4]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,2,26,2024-08-06 18:52:20
1,大安區,21,3,18,2024-08-06 18:51:21
2,大安區,16,2,14,2024-08-06 18:52:20
3,大安區,11,9,2,2024-08-06 18:52:20
4,大安區,16,5,11,2024-08-06 18:51:21
...,...,...,...,...,...
1433,臺大公館校區,30,0,30,2024-08-06 18:52:14
1434,臺大公館校區,35,0,35,2024-08-06 18:35:06
1435,臺大公館校區,24,3,21,2024-08-06 18:47:20
1436,臺大公館校區,40,19,21,2024-08-06 18:52:20


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   行政區域    1438 non-null   object        
 1   總數量     1438 non-null   int64         
 2   可借數量    1438 non-null   int64         
 3   可還數量    1438 non-null   int64         
 4   時間      1438 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 56.3+ KB


In [7]:
df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].sum()

,總數量,可借數量,可還數量
行政區域,,,
中山區,3904,1089,2754
中正區,3267,856,2367
信義區,3360,961,2350
內湖區,3521,1057,2442
北投區,2537,1036,1489
南港區,2317,766,1544
士林區,2962,1276,1601
大同區,1629,615,1008
大安區,4986,1546,3343
